#1

# In a Nim learning model, the environment is the board where there are a certain number of items in three piles. The state is how much is in each stack/pile. In this case, the amount of stacks is 3, and the max items in the stacks is 10 items.  Basically, if they get the last item in the last pile, they lose. 

#2

# The agent in the MIn model is the q learning agent, and essentailly this learning agent is basically trying to make the correct move to get the highest reward. Sometimes, this q learner can also explore the environment and decide between random or going with the high value. Essentially, the q learner will learn the actions, previous state, and the reward. The learner gets expereince from playing over and over and then will know what types of actions to do to get the reward and what not to do. The guru is also a quasi agent although it has just the perfect solution and does nothing else. It is essentially a programed agent with no other purpose.  Nim random is also an agent that makes random moves and carries out exploration 



#3

# The reward for the model essentially grants a reward if they make a move that wins the game where they will get a reward of a positive integer typically, 1 but for this case defined in this case as 100. This is defined in qtable, Alpha, Gamma, Reward = None, 1.0, 0.8, 100.0. Typically the reward is some sort of postive reinforcement that would lead to more moves to obtain this award more often. Then the punishment is a negative integer. Essentially, rewards would usually cause the learner to make more positive moves to get the rewards



#4 and 5

# The number of possible states if there are max 10 items per pile and 3 piles total is (1+10) * (1+10) * (1+10) = 1331

# The possible actions is 10*3 or 30 possible actions



In [1]:
import numpy as np
from random import randint, choice

# The number of piles is 3


# max number of items per pile
ITEMS_MX = 10

# Initialize starting position
def init_game():
    return [randint(1,ITEMS_MX), randint(1,ITEMS_MX), randint(1,ITEMS_MX)]

# Based on X-oring the item counts in piles - mathematical solution
def nim_guru(st):
    xored = st[0] ^ st[1] ^ st[2]
    if xored == 0:
        return nim_random(st)
    for pile in range(3):
        s = st[pile] ^ xored
        if s <= st[pile]:
            return st[pile]-s, pile

# Random Nim player
def nim_random(_st):
    pile = choice([i for i in range(3) if _st[i]>0])  # find the non-empty piles
    return randint(1, _st[pile]), pile  # random move


In [2]:



def nim_qlearner(_st):
    global qtable
    # pick the best rewarding move, equation 1
    a = np.argmax(qtable[_st[0], _st[1], _st[2]])  # exploitation
    # index is based on move, pile
    move, pile = a%ITEMS_MX+1, a//ITEMS_MX
    # check if qtable has generated a random but game illegal move - we have not explored there yet
    if move <= 0 or _st[pile] < move:
        move, pile = nim_random(_st)  # exploration
    return move, pile  # action



In [3]:
Engines = {'Random':nim_random, 'Guru':nim_guru, 'Qlearner':nim_qlearner}

def game(a, b):
    state, side = init_game(), 'A'
    while True:
        engine = Engines[a] if side == 'A' else Engines[b]
        move, pile = engine(state)
        # print(state, move, pile)  # debug purposes
        state[pile] -= move
        if state == [0, 0, 0]:  # game ends
            return side  # winning side
        side = 'B' if side == 'A' else 'A'  # switch sides

def play_games(_n, a, b):
    from collections import defaultdict
    wins = defaultdict(int)
    for i in range(_n):
        wins[game(a, b)] += 1
    # info
    print(f"{_n} games, {a:>8s}{wins['A']:5d}  {b:>8s}{wins['B']:5d}")
    return wins['A'], wins['B']

In [4]:
# Function to print the entire set of states
def qtable_log(_fn):
    with open(_fn, 'w') as fout:
        s = 'state'
        for a in range(ITEMS_MX*3):
            move, pile = a%ITEMS_MX+1, a//ITEMS_MX
            s += ',%02d_%01d' % (move,pile)
        print(s, file=fout)
        for i, j, k in [(i,j,k) for i in range(ITEMS_MX+1) for j in range(ITEMS_MX+1) for k in range(ITEMS_MX+1)]:
            s = '%02d_%02d_%02d' % (i,j,k)
            for a in range(ITEMS_MX*3):
                r = qtable[i, j, k, a]
                s += ',%.1f' % r
            print(s, file=fout)

qtable, Alpha, Gamma, Reward = None, 1.0, 0.8, 100.0


In [5]:

# learn from _n games, randomly played to explore the possible states
def nim_qlearn(_n):
    global qtable
    # based on max items per pile
    qtable = np.zeros((ITEMS_MX+1, ITEMS_MX+1, ITEMS_MX+1, ITEMS_MX*3), dtype=np.float32)
    # play _n games
    for i in range(_n):
        # first state is starting position
        st1 = init_game()
        while True: # while game not finished
        # make a random move - exploration
            move, pile = nim_random(st1)
            st2 = list(st1)
            # make the move
            st2[pile] -= move # --> last move I made
            if st2 == [0, 0, 0]: # game ends
                qtable_update(Reward, st1, move, pile, 0) # I won
                break # new game
    
            elif np.max(qtable[st2[0], st2[1], st2[2]]) >= Reward:
                # immediate loss - penalize it
                qtable_update(-Reward, st1, move, pile, np.min(qtable[st2[0], st2[1], st2[2]]))
    
            else:
                    # not immediate loss - reward it
                qtable_update(Reward, st1, move, pile, np.max(qtable[st2[0], st2[1], st2[2]]))
   
            # Switch sides for play and learning
            
            st1 = st2
 # Equation 3 - update the qtable
def qtable_update(r, _st1, move, pile, q_future_best):
    a = pile*ITEMS_MX+move-1
    qtable[_st1[0], _st1[1], _st1[2], a] = Alpha * (r + Gamma * q_future_best)

In [6]:
nim_qlearn(1000)

In [7]:
# Play games
play_games(1000, 'Random', 'Random')
play_games(1000, 'Guru', 'Random')
play_games(1000, 'Random', 'Guru')
play_games(1000, 'Guru', 'Guru') ;



1000 games,   Random  501    Random  499
1000 games,     Guru 1000    Random    0
1000 games,   Random    5      Guru  995
1000 games,     Guru  949      Guru   51


In [8]:
# Play games
play_games(1000, 'Qlearner', 'Random')
play_games(1000, 'Random', 'Qlearner')

play_games(1000, 'Random', 'Random') ;



1000 games, Qlearner  922    Random   78
1000 games,   Random   80  Qlearner  920
1000 games,   Random  512    Random  488


In [9]:
# See the training size effect
n_train = (3, 10, 100, 1000, 10000, 50000, 100000)
wins = []
for n in n_train:
    nim_qlearn(n)
    a, b = play_games(1000, 'Qlearner', 'Random')
    wins += [a/(a+b)]


1000 games, Qlearner  504    Random  496
1000 games, Qlearner  585    Random  415
1000 games, Qlearner  764    Random  236
1000 games, Qlearner  892    Random  108
1000 games, Qlearner  987    Random   13
1000 games, Qlearner  999    Random    1
1000 games, Qlearner  997    Random    3


In [10]:
# Check the ratio of wins wrt to size of the reinforcement model training
print(wins)



[0.504, 0.585, 0.764, 0.892, 0.987, 0.999, 0.997]


In [11]:
# Function to print the entire set of states
def qtable_log(_fn):
    with open(_fn, 'w') as fout:
        s = 'state'
        for a in range(ITEMS_MX*3):
            move, pile = a%ITEMS_MX+1, a//ITEMS_MX
            s += ',%02d_%01d' % (move,pile)
        print(s, file=fout)
        for i, j, k in [(i,j,k) for i in range(ITEMS_MX+1) for j in range(ITEMS_MX+1) for k in range(ITEMS_MX+1)]:
            s = '%02d_%02d_%02d' % (i,j,k)
            for a in range(ITEMS_MX*3):
                r = qtable[i, j, k, a]
                s += ',%.1f' % r
            print(s, file=fout)

qtable_log('qtable_debug.txt')

